### Clustering using Hash functions
Trying to implement strand isolation using purely Hash functions. Want to select the same strands together after signatures using some similarity metric

In [1]:
%load_ext autoreload
%autoreload 2

In [6]:
import numpy as np
from aligned_clustering import conduct_align_clustering
from utils import get_fastq_records, load_json_file, get_original_strands, get_badread_strand_id, get_recovery_percentage, create_random_strand
import Levenshtein
import random
from Levenshtein import ratio
from collections import Counter
from heirarchal_clustering import filter_junk_reads
from seq_stat import align
import matplotlib.pyplot as plt
import heirarchal_clustering

In [40]:
#records_original = get_fastq_records(r"C:\Users\Parv\Doc\RA\Projects\incomplete_cycles\v2\runs\2025-01-30 21.21.00.463318\reads_no_adapters.fastq")
records_original = get_fastq_records(r"C:\Users\Parv\Doc\RA\Projects\incomplete_cycles\v2\runs\2025-01-30 21.21.00.463318\reads_adapters.fastq\reads_adapters.fastq")

In [41]:
original_strand_ids, coupling_rates, capping_flags, original_strands = get_original_strands(r"C:\Users\Parv\Doc\RA\Projects\incomplete_cycles\v2\runs\2025-01-30 21.21.00.463318\original_strands.txt")

strand_ids_synthesized = load_json_file(r"C:\Users\Parv\Doc\RA\Projects\incomplete_cycles\v2\runs\2025-01-30 21.21.00.463318\synthesized_uid_reference.json")

In [44]:
records = [i for i in records_original if get_badread_strand_id(i) in strand_ids_synthesized]
#records = filter_junk_reads(records)
seqs = [str(i.seq) for i in records]

In [45]:
len(seqs)

287

In [65]:
clusters = heirarchal_clustering.cluster_trivial(seqs, similarity_threshold=0.9)

287it [00:00, 40989.01it/s]


In [66]:
len(clusters)

17

Levenshtien edit distance works pretty well. That's my metric. Kmeans works for now. I'll bring down junk reads etc, remove adapters, and test it again to see how well aggregation works.
Maybe try DBscan with Levenshtien distance

In [71]:

clustered_seqs = []
for j in clusters:
    clustered_seqs.append([str(records[ind].seq) for ind in clusters[j]])

In [75]:
clustered_seqs.sort(key=len, reverse=True)
top_5 = clustered_seqs[:5]

In [78]:
# And then merge and validate for top 5

for i in range(len(top_5)):
    guess = heirarchal_clustering.make_prediction(clustered_seqs[i], sample_size=15)
    for k, original_strand in enumerate(original_strands):
        rec = get_recovery_percentage(guess, original_strand)
        print(f"{k} {rec} recovered by {i}".format())
    print()

0 0.22 recovered by 0
1 0.25 recovered by 0
2 0.305 recovered by 0
3 0.26 recovered by 0
4 0.225 recovered by 0

0 0.3 recovered by 1
1 0.24 recovered by 1
2 0.26 recovered by 1
3 0.215 recovered by 1
4 0.275 recovered by 1

0 0.28 recovered by 2
1 0.235 recovered by 2
2 0.29 recovered by 2
3 0.26 recovered by 2
4 0.28 recovered by 2

0 0.32 recovered by 3
1 0.24 recovered by 3
2 0.28 recovered by 3
3 0.315 recovered by 3
4 0.2 recovered by 3

0 0.26 recovered by 4
1 0.255 recovered by 4
2 0.245 recovered by 4
3 0.275 recovered by 4
4 0.255 recovered by 4



In [79]:
# Needs further testing